<a href="https://colab.research.google.com/github/Felipe34515/Trabajos-BI/blob/main/Proyecto_Anal%C3%ADtica_de_textos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Proyecto Analítica de textos**

## Integrantes:
- Felipe Rueda - 202010903
- Santiago Pardo - 202013025
- Luis Plazas - 202013155

## Contexto:

El Ministerio de Comercio, Industria y Turismo de Colombia, la Asociación Hotelera y
Turística de Colombia – COTELCO, cadenas hoteleras de la talla de Hilton, Hoteles Estelar,
Holiday Inn y hoteles pequeños ubicados en diferentes municipios de Colombia están
interesados en analizar las características de sitios turísticos que los hacen atractivos para
turistas locales o de otros países, ya sea para ir a conocerlos o recomendarlos. De igual
manera, quieren comparar las características de dichos sitios, con aquellos que han
obtenido bajas recomendaciones y que están afectando el número de turistas que llegan a
ellos. Adicionalmente, quieren tener un mecanismo para determinar la calificación que
tendrá un sitio por parte de los turistas y así, por ejemplo, aplicar estrategias para identificar
oportunidades de mejora que permitan aumentar la popularidad de los sitios y fomentar el
turismo.

Esos actores de turismo prepararon dos conjuntos de datos con reseñas de sitios turísticos.
Cada reseña tiene una calificación según el sentimiento que tuvo el turista al visitarlo. Estos
actores quieren lograr un análisis independiente de los conjuntos de datos y al final del
proyecto discutir sobre los grupos de científicos de datos e ingenieros de datos que
acompañarán el desarrollo real de este proyecto.






In [26]:
# @title Librerías a utilizar

import pandas as pd
import matplotlib.pyplot as plt
import joblib
from joblib import dump, load
import numpy as np

import spacy
import nltk
from nltk.corpus import stopwords
import torch
import torchtext

In [22]:
# @title Instalar dependencias (Solo instalar una vez)

# !pip install spacy
# !python -m spacy download es_core_news_sm
# !pip install nltk


In [18]:
# @title Carga de datos
df = pd.read_csv("tipo1_entrenamiento_estudiantes.csv", delimiter=",")
df

,Review,Class
0,Nos alojamos en una casa alquilada en la ciuda...,4
1,"La comida está bien, pero nada especial. Yo te...",3
2,"En mi opinión, no es una como muchos usuarios ...",3
3,esta curiosa forma que asemeja una silla de mo...,4
4,Lo mejor era la limonada. Me gusto la comida d...,2
...,...,...
7870,El motivo de mi estancia fue porque vine a un ...,3
7871,Es difícil revisar el castillo porque apenas p...,3
7872,Si vas a Mérida no puedes perderte de este lug...,5
7873,"Este imperdible sitio, que lleva el nombre del...",5


# **(10%) Entendimiento del negocio y enfoque analítico.**

### Objetivos y criterios de éxito

### Enfoque analítico




# **(20%) Entendimiento y preparación de los datos.**

### Quitar las Stop Words

Las "stop words" en un modelo de procesamiento de lenguaje natural (NLP, por sus siglas en inglés) son palabras que se filtran antes o después del procesamiento del texto. Cuando construimos el vocabulario de un modelo, las "stop words" son las palabras que excluimos. Generalmente, estas palabras son muy comunes en el lenguaje (como "en", "y", "a", "el", "la", etc. en español) y no contienen información importante para entender el contexto de un texto. Por lo tanto, se eliminan para reducir la cantidad de ruido en los datos y mejorar la eficiencia del modelo. Sin embargo, en algunos casos, como en el análisis de sentimientos, las "stop words" pueden ser útiles y se mantienen.

In [19]:
# @title Quitar Stop Words
nltk.download("stopwords")
stop_words = set(stopwords.words("spanish"))
df["Review"] = df["Review"].apply(lambda row: " ".join([word for word in row.split() if word.lower() not in stop_words]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Lematización

La lematización es un proceso en el procesamiento del lenguaje natural que consiste en reducir las palabras a su forma base o raíz, conocida como "lema". Por ejemplo, las palabras "corriendo", "corrió" y "correrá" se lematizan a "correr".

La lematización ayuda a reducir la dimensionalidad de los datos de texto y a agrupar palabras similares, lo que puede mejorar la eficacia de muchos algoritmos de NLP.

In [23]:
# @title Lematizar
nlp = spacy.load("es_core_news_sm")
df["Review"] = df["Review"].apply(lambda row: " ".join([token.lemma_ for token in nlp(row)]))

In [24]:
df

,Review,Class,Review_lematizado
0,alojamo casa alquilado ciudad amurallado . par...,4,alojamo casa alquilado ciudad amurallado . par...
1,"comida bien , especial . mejor comida Mexcan U...",3,"comida bien , especial . mejor comida Mexcan U..."
2,"opinión , usuario reclamar . gran paladar pare...",3,"opinión , usuario reclamar . gran paladar pare..."
3,curioso forma asemejo silla montar ahi nombre ...,4,curioso forma asemejo silla montar ahi nombre ...
4,mejor limonado . gusto comida mundo sós frío .,2,mejor limonado . gusto comida mundo sós frío .
...,...,...,...
7870,"motivo estancia venir congreso medico , hosped...",3,"motivo estancia venir congreso medico , hosped..."
7871,difícil revisar castillo apenas poder caminar ...,3,difícil revisar castillo apenas poder caminar ...
7872,si ir mérido poder perderte lugar nuevo sucurs...,5,si ir mérido poder perderte lugar nuevo sucurs...
7873,"imperdible sitio , llevar nombre conquistador ...",5,"imperdible sitio , llevar nombre conquistador ..."


### Tokenización

La tokenización es el proceso de sustituir los datos sensibles por símbolos de identificación únicos que conservan toda la información esencial de los datos sin comprometer su seguridad.

In [29]:
TEXT =  torchtext.data.Field(tokenize = 'spacy')
LABEL = torchtext.data.LabelField(dtype = torch.long)

TEXT.build_vocab(df, max_size = 10000)
LABEL.build_vocab(df)

AttributeError: module 'torchtext.data' has no attribute 'Field'

# **(20%) Modelado y evaluación.**

# **(20%) Resultados.**

# **(10%) Mapa de actores relacionado con el producto de datos creado.**

# **(8%) Trabajo en equipo**